# Ranking Words

## Define factorial

In [35]:
from functools import reduce
def fac(n):
    return reduce(lambda x,y: x*y, range(1,n+1))

## Binomial Coefficient

We use recursion: 

$${n\choose m} = \frac{n}{m}{n-1\choose m-1}$$



In [36]:
def binomial(n, m):
    if n == 0:
        return 0
    if m == 0:
        return 1
    return (binomial(n-1, m-1)*n) // m

## Multinomial Coefficient

A *multinomial coefficient* tell us the number of multi-permutations of a certain kind. For example, the number of multi-permutations of 3 letters A, 2 letters B, and 1 letter C, is:

$${6 \choose 3,2,1} = \frac{6!}{3! 2! 1!} = 60$$

We use the recurrence:

$${n \choose n_1, n_2, \cdots,n_k} = {n_1+n_2\choose n_1} {n\choose n_1+n_2, n_3, \cdots , n_k}$$

where $n=n_1+n_2+\cdots+n_k$.

In [37]:
def multinomial(nums):
    if len(nums) <= 1:
        return 1
    a, b, *rest = nums
    return binomial(a+b, a) * multinomial([a+b] + rest)

For example:

In [38]:
multinomial([3, 2, 1])

60

For a multi-permutation with **no** repeated letters, the multinomial coefficient must coincide with the factorial:

In [39]:
multinomial([1,1,1,1]) == fac(4)

True

## Rank of an element

Given an arbitrary list (permutation or multi-permutation), find the rank of an element:

In [40]:
def rank(x, perm):
    return len([i for i in set(perm) if i < x])

For example, the rank of 7 in `[2, 5, 7, 3, 10]` should be 3 (rank starts at 0):

In [41]:
rank(7, [2, 5, 7, 3])

3

## Rank of a multi-permutation

`decrease_kind` is a helper function that finds the kind of a multi-permutation when we remove one of its elements.  For example: consider a multi-permutation of `[0,0,1,1,2]`, with kind $(2,2,1)$. If we remove one `0`, it becomes a multi-permutation of kind $(1,2,1)$, but if we remove the `2`, its kind becomes $(2,2)$.

In [42]:
def decrease_kind(kind, i):
    new_kind = list(kind)
    new_kind[i] -= 1
    return [u for u in new_kind if u]

In [43]:
print(decrease_kind([2,2,1], 0))
print(decrease_kind([2,2,1], 2))

[1, 2, 1]
[2, 2]


We compute the rank of a multi-permutation with a recursion. 

Assume first we have a permutation $p$ of $n$ elements.  If we already know how to compute ranks of permutations with $n-1$ elements, then the rank of $p$ is

$$R_n(p) = r (n-1)! + R_{n-1}(p[1{:}])$$

where $r$ is the rank of $p[0]$.

For a multi-permutation $p$ of kind $(n_1, n_2, \ldots, n_k)$, the recursion is:

$$R_{n_1,n_2,\ldots,n_k}(p) = \sum_{i=1}^{r-1}{n\choose n_1, \ldots, n_i-1, \ldots, n_k} + R_{n_1, \ldots, n_r-1,\ldots,n_k}(p[1{:}])$$

In [44]:
def multiperm_rank(multiperm, kind):
    if not multiperm:
        return 0
    x, *rest = multiperm
    r = rank(x, multiperm)
    c = sum(multinomial(decrease_kind(kind, i)) for i in range(r))
    return c + multiperm_rank(rest, decrease_kind(kind, r))

Now we do it for words:

In [45]:
from collections import Counter

def word_to_multiperm(word):
    dic = {y:x for x, y in enumerate(sorted(''.join(set(word))))}
    return [dic[c] for c in word]

def find_kind(word):
    return [y for _, y in sorted(Counter(word).most_common())]
    
def word_rank(word):
    kind = find_kind(word)
    multiperm = word_to_multiperm(word)
    return multiperm_rank(multiperm, kind) + 1

###Main function

Now let's test it...

In [46]:
words = [
    'BOOKKEEPER',
    'ABBA',
    'INDOCRINATE'
    'THISISTWENTYFIVELETTERSSS'
]

for word in words:
    print('Testing %s, length %d:' % (word, len(word)))
    %time rank = word_rank(word)
    print('Rank: %d' % (rank))
    
    

Testing BOOKKEEPER, length 10:
CPU times: user 103 µs, sys: 43 µs, total: 146 µs
Wall time: 148 µs
Rank: 10743
Testing ABBA, length 4:


TypeError: 'int' object is not callable

In [ ]:
%time word_rank('ABBA')